In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import firedrake
import icepack, icepack.models, icepack.plot

# Synthetic ice sheet

In this demo we'll run an experiment inspired by the paper [*Fjord insertion into continental margins
driven by topographic steering of ice*](https://www.nature.com/articles/ngeo201) by Kessler and others.
This work simulated the evolution of an entire ice sheet on millenial timescales, with the added twist that the bedrock topography could evolve under the influence of erosion.

### Geometry

The model domain is a circle 250 km wide, but with a 1km-high ridge at a radius of 200 km.
This ridge is punctuated by four valleys of varying depths from which ice streams form.

In [ ]:
import pygmsh

R = 250e3
δx = 25e3

geometry = pygmsh.built_in.Geometry()

points = [geometry.add_point([+R, 0, 0], lcar=δx),
          geometry.add_point([0, +R, 0], lcar=δx),
          geometry.add_point([-R, 0, 0], lcar=δx),
          geometry.add_point([0, -R, 0], lcar=δx)]
center = geometry.add_point([0, 0, 0], lcar=δx)

arcs = [geometry.add_circle_arc(points[k], center, points[(k + 1) % len(points)])
        for k in range(len(points))]

line_loop = geometry.add_line_loop(arcs)
plane_surface = geometry.add_plane_surface(line_loop)

physical_lines = [geometry.add_physical(arc) for arc in arcs]
physical_surface = geometry.add_physical(plane_surface)

Now we're going to do some fancy tricks to make the mesh more refined around the mountain range.
See gmsh example 10.

In [ ]:
code = geometry.get_code()

code += """
lc = {1:};

Field[1] = Distance;
Field[1].NNodesByEdge = 100;
Field[1].EdgesList = {{{0:}}};

Field[2] = Threshold;
Field[2].IField = 1;
Field[2].LcMin = lc / 30;
Field[2].LcMax = lc;
Field[2].DistMin = 1e3;
Field[2].DistMax = 50e3;

Background Field = 2;

Mesh.CharacteristicLengthExtendFromBoundary = 0;
Mesh.CharacteristicLengthFromPoints = 0;
Mesh.CharacteristicLengthFromCurvature = 0;
""".format(', '.join([arc.id for arc in arcs]), δx)

print(code)

with open('ice-sheet.geo', 'w') as geo_file:
    geo_file.write(code)

In [ ]:
!gmsh -2 -format msh2 -o ice-sheet.msh ice-sheet.geo

In [ ]:
mesh = firedrake.Mesh('ice-sheet.msh')

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh)